In [2]:
import os
import faiss
import json

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
from langchain.schema import Document
from uuid import uuid4
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


In [3]:
os.environ["OpenAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model ="gpt-4o")

In [4]:
with open("/Users/t2023-m0072/Desktop/AI_8_CH-3_LLM-RAG_AI_Utilizatioon_App/personal_work/차아인/food_recipes.json",'r',encoding='utf-8') as file:
    data = json.load(file)

In [5]:
document_recipe = [
    Document(page_content=str(recipe), metadata={"요리명": recipe["요리명"],"요리재료" : recipe["요리재료"], "기본정보" : recipe["기본정보"], "조리순서" : recipe["조리순서"]})
    for recipe in data
]

In [6]:
print(document_recipe)

[Document(metadata={'요리명': '미역국', '요리재료': [{'재료': '미역(마른것)', '용량': '5줌(20g)'}, {'재료': '쇠고기(양지머리)', '용량': '120g'}, {'재료': '물', '용량': '8컵(1'}, {'재료': '재래간장 1과', '용량': '1/2큰술(22ml)'}, {'재료': '마늘(다진 마늘)', '용량': '1큰술(10g)'}, {'재료': '소금', '용량': '작은술(3g)'}, {'재료': '참기름', '용량': '작은술(5ml)'}], '기본정보': {'조리시간': '30분', '분량': '4인분 기준', '칼로리': '81kcal (1인분)'}, '조리순서': ['01. 마른 미역은 찬물에 담가 10분간 불린다. 찬물에 바락바락 씻어 거품이 나오지 않을 때까지 헹군다.', '02. 물기를 꼭 짠 후 적당한 크기로 자른 후 재래간장 1/2큰술을 넣고 조물조물 무친다.', '03. 쇠고기는 한입 크기로 썬 후 달군 냄비에 참기름을 두르고 쇠고기, 마늘을 넣어 볶다가 쇠고기가 거의 익으면 미역을 넣고 볶는다.', '04. 03에 물을 넣고 한소끔 끓인다. 재래간장과 소금으로 간하고 더 끓인다. (물 대신 쌀뜨물을 넣으면 더욱 구수하고 맛있는 미역국을 만들 수 있다.)']}, page_content="{'요리명': '미역국', '요리재료': [{'재료': '미역(마른것)', '용량': '5줌(20g)'}, {'재료': '쇠고기(양지머리)', '용량': '120g'}, {'재료': '물', '용량': '8컵(1'}, {'재료': '재래간장 1과', '용량': '1/2큰술(22ml)'}, {'재료': '마늘(다진 마늘)', '용량': '1큰술(10g)'}, {'재료': '소금', '용량': '작은술(3g)'}, {'재료': '참기름', '용량': '작은술(5ml)'}], '기본정보': {'조리시간': '30분', '분량': '4인분 기준', '칼로리': '81kcal (1인분)'}, '조리순서': [

In [7]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

In [8]:
index  = faiss.IndexFlatL2(len(embeddings.embed_query("레시피")))

recipe_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [9]:
uuids = [str(uuid4()) for _ in range(len(document_recipe))]

recipe_store.add_documents(documents=document_recipe, ids = uuids)

['1f99a1b6-1f41-49d3-aabd-5826f3ea86f1',
 '6ad8c704-5ff0-4cde-aaaf-22b85e570042',
 '48fabc2e-190d-4d6d-b303-89cddbefd1d3',
 '48861ee1-bdf5-4bbe-90ce-9c6450dcd8e1',
 'e5be6e74-943a-4778-9113-663cb47500f2',
 'd693f8ae-7413-45e1-b82b-8d0c29cb3c2f',
 'a5ef6ae8-1835-4674-8d27-c15058fdf334',
 'd7179fcd-5898-4aab-9ba5-082b82fdbd64',
 'a4fe86c6-1901-4f91-a243-882db0126ef3',
 '34553d18-70da-4a8f-a1f9-68433d63fa5d']

In [10]:
retriever = recipe_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [24]:
'''prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 주어진 데이터로만 답변을 할 수 있어."),
    ("system", "너는 요리의 전문가야."),
    ("system", "질문에 답변할 때는 항상 데이터를 세밀히 학습하고 정확한 답변을 생성해야 해."),
    ("system", "만약 데이터 안에 유저의 질문이 없다면, 양해를 구하고 관련 데이터를 제공할 수 없음을 알리도록 해."),
    ("system", "질문에 계산이 필요한 경우, 데이터를 기반으로 정확하게 계산하여 결과를 제공해야 합니다."),
    ("system", "질문에 대한 답변을 생성 하기전에 검증을 마친 후에 생성해줘."),
    ("system", "부족한 부분이 있다고 생각하면, 다른 데이터를 참조해줘"),

    
    ("system", "다음은 답변 형식의 예시야: \n"
               "user: 미역국을 만들고 싶어.\n"
               "ai: 미역국의 재료는 ~입니다.\n"
               "ai: 미역국을 만드는 순서는 다음과 같습니다:\n"
               "ai: 첫 번째 ~~~, 두 번째 ~~~입니다.\n"),
    
    ("system", "또 다른 예시를 들어줄게:\n"
               "user: 내가 만들려고 하는 미역국의 칼로리는 얼마야?\n"
               "ai: 미역국의 칼로리는 1인분당 ~kcal입니다. 이 레시피는 ~인분 기준이므로, 총 칼로리는 ~kcal입니다.\n"),
    
    ("system", "추가 예시:\n"
               "user: 5인분의 레시피로 수정해줘.\n"
               "ai: 현재 미역국 레시피는 ~인분 기준입니다. 5인분으로 수정된 재료는 다음과 같습니다:\n"
               "ai: ~~~.\n"
               "이때도 순서를 전부 알려줘"),
    
    ("user", "다음과 같은 데이터를 학습해:\n{data}"),
    ("user", "그리고 질문에 답해:\n{question}")
])'''

'prompt = ChatPromptTemplate.from_messages([\n    ("system", "너는 주어진 데이터로만 답변을 할 수 있어."),\n    ("system", "너는 요리의 전문가야."),\n    ("system", "질문에 답변할 때는 항상 데이터를 세밀히 학습하고 정확한 답변을 생성해야 해."),\n    ("system", "만약 데이터 안에 유저의 질문이 없다면, 양해를 구하고 관련 데이터를 제공할 수 없음을 알리도록 해."),\n    ("system", "질문에 계산이 필요한 경우, 데이터를 기반으로 정확하게 계산하여 결과를 제공해야 합니다."),\n    ("system", "질문에 대한 답변을 생성 하기전에 검증을 마친 후에 생성해줘."),\n    ("system", "부족한 부분이 있다고 생각하면, 다른 데이터를 참조해줘"),\n\n    \n    ("system", "다음은 답변 형식의 예시야: \n"\n               "user: 미역국을 만들고 싶어.\n"\n               "ai: 미역국의 재료는 ~입니다.\n"\n               "ai: 미역국을 만드는 순서는 다음과 같습니다:\n"\n               "ai: 첫 번째 ~~~, 두 번째 ~~~입니다.\n"),\n    \n    ("system", "또 다른 예시를 들어줄게:\n"\n               "user: 내가 만들려고 하는 미역국의 칼로리는 얼마야?\n"\n               "ai: 미역국의 칼로리는 1인분당 ~kcal입니다. 이 레시피는 ~인분 기준이므로, 총 칼로리는 ~kcal입니다.\n"),\n    \n    ("system", "추가 예시:\n"\n               "user: 5인분의 레시피로 수정해줘.\n"\n               "ai: 현재 미역국 레시피는 ~인분 기준입니다. 5인분으로 수정된 재료는 다음과 같습니다:\n"\n               

In [25]:

prompt = ChatPromptTemplate.from_template("""
너는 사용자가 입력한 [재료]에 따라 관련 요리를 알려주는 요리사야. 
친절하고 전문적인 말투로 응답해.
이 재료들로 만들 수 있는 요리 최대 5개를 가져온 데이터에 해당하는 음식을 우선적으로 알려줘.이 때 각각 사용되는 재료와 함께 알려줘.

아래는 대화의 예시야
'''
user : 미역으로 만들 수 있는 음식 4개를 알려줘
ai : 미역이 재료로 포함되어 만들 수 있는 음식은 아래와 같습니다.

- 음식명1
    - 필요 재료 나열
- 음식명2
    - 필요 재료 나열
- 음식명3
    - 필요 재료 나열
- 음식명4
    - 필요 재료 나열
'''


데이터에 포함되어 있는 레시피를 우선적으로 제공하고, 데이터에 포함되어 있지 않은 레시피의 경우에는 일반적인 레시피를 알려주도록해.
기본적으로 해당 레시피의 원래 인분 수로 제공하고, 사용자가 원하는 인분 수에 따라 재료의 양을 계산해서 수정해줘.
데이터에 유저의 질문이 포함되어 있지 않다면, 양해를 구하고 관련 데이터를 제공할 수 없음을 말해줘.

'''
user : <음식명3> 을 만드는 법을 알려줘
ai : <음식명3> 을 만드는 방법은 아래와 같습니다.
- 요리의 이름
- 재료
    - 재료 1 (재료양)
    - 재료 2 (재료양)
    - 재료 3 (재료양)
- 조리 시간
- 인분 수
- 요리 방법
    - 첫번째, ~~
    - 두번째, ~~
    - 세번째, ~~
'''

'''
user : 1인분의 레시피를 알려줘
ai : <음식명>의 레시피는 ~ 인분 기준입니다. 
<음식명> 을 1인분으로 만드는 레시피는 다음과 같습니다
- 재료 : 1인분으로 수정된 재료 나열
- 요리 방법 : 요리 방법 나열
'''

다음과 같은 데이터를 학습해:\n{data}"
그리고 질문에 답해:\n{question}

""")

In [26]:
contextual_prompt = prompt

In [27]:
class DebugPassThrough(RunnablePassthrough):
    
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args,**kwargs)
        return output
    
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config = None, **kwargs):
        data_text = inputs
        
        return {"data": data_text, "question": inputs["question"]}
    
rag_chain_debug = {
    "data" : retriever,
    "question" : DebugPassThrough()
    
} | DebugPassThrough() | ContextToText()| contextual_prompt | model

In [28]:
while True:
    print("--------------------")
    
    query = input("질문을 입력해 주세요 (break 입력시 종료됩니다) : ")
    
    if query.lower() == "break":
        break
    
    response = rag_chain_debug.invoke(query)
    
    print("Question : ", query)
    print(response.content)

--------------------
Question :  김치로 만들 수 있는 요리를 알려줘
김치가 재료로 포함되어 만들 수 있는 요리는 아래와 같습니다.

1. 김치찌개
   - 필요 재료: 김치, 돼지고기, 두부, 대파, 마늘, 고춧가루, 물, 소금, 간장

2. 김치볶음밥
   - 필요 재료: 김치, 밥, 대파, 돼지고기 또는 햄, 간장, 참기름, 깨

3. 김치전
   - 필요 재료: 김치, 부침가루, 물, 대파, 식용유

4. 김치만두
   - 필요 재료: 김치, 돼지고기, 두부, 부추, 만두피, 간장, 참기름, 마늘

5. 김치우동
   - 필요 재료: 김치, 우동면, 돼지고기 또는 해산물, 대파, 간장, 참기름, 물

이 요리들은 김치를 활용하여 맛있게 즐길 수 있는 대표적인 메뉴들입니다. 각 요리의 세부 조리 방법이 필요하시다면 말씀해 주세요.
--------------------
Question :  김치찌개 레시피를 알려줘
죄송하지만, 현재 제공된 데이터에는 김치찌개의 레시피가 포함되어 있지 않습니다. 일반적인 김치찌개의 레시피를 알려드릴 수 있습니다.

**김치찌개** 레시피는 다음과 같습니다:

- **재료**
  - 김치 300g
  - 돼지고기(삼겹살 또는 목살) 200g
  - 두부 1모
  - 대파 1대
  - 양파 1/2개
  - 물 600ml
  - 고춧가루 1큰술
  - 간장 1큰술
  - 다진 마늘 1큰술
  - 소금, 후추 약간

- **조리 시간**
  - 약 30분

- **인분 수**
  - 3~4인분 기준

- **요리 방법**
  1. 김치는 먹기 좋은 크기로 자르고, 돼지고기는 한 입 크기로 썰어둡니다.
  2. 냄비에 김치와 돼지고기를 넣고 고춧가루, 간장, 다진 마늘을 추가한 후 중불에서 5분 정도 볶아줍니다.
  3. 물을 붓고 끓기 시작하면 불을 줄여 15분 정도 끓입니다.
  4. 두부는 먹기 좋은 크기로 썰고, 양파와 대파도 슬라이스 해줍니다.
  5. 두부, 양파, 대파를 넣고 소금과 후추로 간을 